# Preprocessing .mat for NWB

In [1]:
import numpy as np
import h5py
import os
import yaml
import pandas as pd
from datetime import datetime
import shutil
import importlib
import numpy as np
import pandas as pd
import convert_to_nwb_for_PL
import converters.Initiation_nwb
import scipy.io


# Load data 

## .mat files

In [101]:
csv_file = "data/Subject_Session_Selection.csv"
PL = "data/mouse/General"
PLALL = "data/mouse/Recording"
PLALL_number1 = "data/mouse/Recording/PL200_D1.mat"
output_folder = "data"

In [12]:
PL200 = [os.path.join(PL, f) for f in os.listdir(PL)][0]
PL200


'data/mouse/General/PL200.mat'

In [89]:
for file_name in sorted(os.listdir(PLALL)):
    i += 1
    file_path = os.path.join(PLALL, file_name)
    if os.path.isfile(file_path) and file_name.endswith('.mat'):
        print(f"Processing file: {file_name}")

Processing file: PL200_D1.mat
Processing file: PL200_D2.mat
Processing file: PL200_D3.mat
Processing file: PL200_D4.mat
Processing file: PL200_D5.mat
Processing file: PL200_D6.mat
Processing file: PL200_D7.mat
Processing file: PL200_D8.mat


### General

In [13]:
from scipy.io import loadmat
General_data = loadmat(PL200)

In [14]:
type(General_data)

dict

In [15]:
print(General_data.keys())


dict_keys(['__header__', '__version__', '__globals__', 'LFP_Data', 'LFP_Data_description'])


In [20]:
print(General_data["__header__"])


b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Mon Dec 18 12:21:35 2017'


In [21]:
print(General_data["__version__"])


1.0


In [22]:
print(General_data["__globals__"])


[]


In [16]:

print(General_data["LFP_Data"])



[[(array([[array(['PL200'], dtype='<U5')],
         [array(['PL200'], dtype='<U5')],
         [array(['PL200'], dtype='<U5')],
         ...,
         [array(['PL200'], dtype='<U5')],
         [array(['PL200'], dtype='<U5')],
         [array(['PL200'], dtype='<U5')]], dtype=object), array([[array(['C57BL/6JRj'], dtype='<U10')],
         [array(['C57BL/6JRj'], dtype='<U10')],
         [array(['C57BL/6JRj'], dtype='<U10')],
         ...,
         [array(['C57BL/6JRj'], dtype='<U10')],
         [array(['C57BL/6JRj'], dtype='<U10')],
         [array(['C57BL/6JRj'], dtype='<U10')]], dtype=object), array([[array(['m'], dtype='<U1')],
         [array(['m'], dtype='<U1')],
         [array(['m'], dtype='<U1')],
         ...,
         [array(['m'], dtype='<U1')],
         [array(['m'], dtype='<U1')],
         [array(['m'], dtype='<U1')]], dtype=object), array([[array([[array([], dtype='<U1')]], dtype=object)],
         [array([[array([], dtype='<U1')]], dtype=object)],
         [array([[array([],

In [17]:
# Mouse Name
print(General_data["LFP_Data"][0][0][0][0][0][0])


PL200


In [ ]:
# id session 
print(General_data["LFP_Data"][0][0][4])
unique_values, unique_indices = np.unique(General_data["LFP_Data"][0][0][4], return_index=True)
print("Unique values:", unique_values)
print("Indices:", unique_indices)



[[1]
 [1]
 [1]
 ...
 [8]
 [8]
 [8]]
Unique values: [1 2 3 4 5 6 7 8]
Indices: [   0  164  332  429  516  695  921 1123]


In [75]:
i=0
# Start date (dd.mm.yy)
print(General_data["LFP_Data"][0][0][5][unique_indices[1]])

# Start time (hhmmss)
## hh
hh = str(General_data["LFP_Data"][0][0][5][unique_indices[i]][3][0][0])
## mm
if len(str(General_data["LFP_Data"][0][0][5][unique_indices[i]][4][0][0])) == 1:
    mm= "0" + str(General_data["LFP_Data"][0][0][5][unique_indices[i]][4][0][0])
else:
    mm = str(General_data["LFP_Data"][0][0][5][unique_indices[i]][4][0][0])
## ss
ss = str(General_data["LFP_Data"][0][0][5][unique_indices[i]][5][0][0])

start_time = hh + mm +  ss[:2]
print("Start time:", start_time)


#Behavior Type
print(str(General_data["LFP_Data"][0][0][6][unique_indices[i]][0][0]))


[array([[2014]], dtype=uint16) array([[6]], dtype=uint8)
 array([[20]], dtype=uint8) array([[15]], dtype=uint8)
 array([[31]], dtype=uint8) array([[20.0009999]])]
Start time: 142055
DT


In [ ]:
# Strain
print(General_data["LFP_Data"][0][0][1][0][0][0])

C57BL/6JRj


In [36]:
# sex 
print(General_data["LFP_Data"][0][0][2][0][0][0])

m


In [ ]:
# birth date 
print(str(General_data["LFP_Data"][0][0][3][0][0][0][0]))

[]


In [165]:
# trial onset 
rien = unique_indices[1]
trial = np.asarray(General_data["LFP_Data"][0][0][8]).flatten()
trialonset_avant= trial[0:rien]

In [ ]:

print(General_data["LFP_Data_description"])

### ALL (number1)

In [102]:
from scipy.io import loadmat
pli = loadmat(PLALL_number1)

In [103]:
pli.keys()

dict_keys(['__header__', '__version__', '__globals__', 'Catch_times', 'EMG', 'PtA', 'Stim_times', 'Valve_times', 'dCA1', 'lick', 'mPFC', 'wM1', 'wS1'])

In [209]:
# Ws2
if "wM1" in pli.keys():
    wM1= np.asarray(pli["wM1"][0][0][1]).flatten()
else:
    print("wM1 not found in pli")


{'__header__': b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Wed Feb  1 10:30:44 2017',
 '__version__': '1.0',
 '__globals__': [],
 'Catch_times': array([[(array([[  47643,   72935,  121426,  145417,  163214,  184412,  237399,
                  262991,  278690,  301383,  392563,  549826,  605312,  625309,
                  711686,  735682,  763476,  983423, 1041709, 1066306, 1157984,
                 1178979, 1199075, 1232265, 1260158, 1412924, 1795438, 1898712,
                 1934504, 1960896, 1988790, 2303217, 2402195, 2474680, 2596551,
                 2630341, 2661533, 2852645, 2990011, 3039001, 3161872, 3192166,
                 3282244, 3409516, 3473301, 3556281, 3654294, 3892839, 3951923,
                 3985515, 4418116, 4484500, 4520590, 4621968, 4792927, 4865714,
                 4946893, 5010177, 5060567, 5211033, 5257920, 5303209, 5337003,
                 5356998, 5419182, 5597643, 5712114, 5763703, 5811494, 5868678,
                 5997548, 6040740, 6077431, 61

In [ ]:
# Stim_times
stim_onset= np.asarray(pli["Stim_times"][0][0][1][0])/1000
# Catch_times
catch_onset = np.asarray(pli["Catch_times"][0][0][0][0])/1000
# trial_onset
all_onsets = np.concatenate([stim_onset, catch_onset])
all_onsets_sorted = np.sort(all_onsets)
print(all_onsets_sorted)
len(all_onsets_sorted)



[  25751.   47643.   72935.   95532.  121426.  145417.  163214.  184412.
  211808.  237399.  262991.  278690.  301383.  318882.  339278.  392563.
  415958.  443154.  477842.  521035.  549826.  566322.  589317.  605312.
  625309.  655801.  711686.  735682.  763476.  791072.  816367.  892048.
  949035.  983423. 1041709. 1066306. 1092102. 1157984. 1178979. 1199075.
 1232265. 1260158. 1304852. 1335645. 1412924. 1436420. 1562292. 1581389.
 1607481. 1664868. 1732354. 1795438. 1862124. 1898712. 1934504. 1960896.
 1988790. 2010387. 2053780. 2128161. 2226039. 2281924. 2303217. 2402195.
 2474680. 2499776. 2574958. 2596551. 2630341. 2661533. 2702428. 2780511.
 2804760. 2852645. 2881941. 2925330. 2949523. 2990011. 3039001. 3066294.
 3161872. 3192166. 3221862. 3282244. 3310340. 3409516. 3473301. 3503993.
 3530387. 3556281. 3615508. 3654294. 3695786. 3729378. 3892839. 3951923.
 3985515. 4016811. 4063602. 4110189. 4244259. 4298647. 4338235. 4399721.
 4418116. 4484500. 4520590. 4554985. 4621968. 46884

180

In [171]:
Valve_times= np.asarray(pli["Valve_times"][0][0][1][0])/1000
Valve_times

array([ 417.655,  478.939,  590.514,  656.5  ,  791.869,  817.26 ,
        892.845,  949.632, 1093.597, 1306.749, 1336.34 , 1437.117,
       1563.188, 1581.986, 1608.077, 1665.267, 1733.151, 1863.021,
       2010.884, 2054.273, 2128.857, 2226.735, 2282.421, 2500.272,
       2575.453, 2703.025, 2781.006, 2882.438, 2925.825, 2950.02 ,
       3066.894, 3222.657, 3311.134, 3504.491, 3530.886, 3615.999,
       3696.383, 3729.874, 4017.508, 4064.099, 4110.686, 4245.154,
       4299.144, 4338.732, 4400.218, 4555.482, 4688.954, 4725.344,
       4891.806, 4976.884, 5095.759, 5156.143, 5233.928, 5275.516,
       5376.194, 5398.888, 5477.669, 5628.034, 5669.824, 5736.109,
       5893.374, 5932.865, 5959.258, 6137.116, 6211.728, 6318.301,
       6445.375, 6471.568, 6531.152, 6614.835, 6707.813, 6763.3  ,
       6893.87 , 7017.842])

In [168]:
# Compare trialonset_avant and all_onsets_sorted
trialonset_avant_sorted = np.sort(trialonset_avant)
are_equal = np.array_equal(trialonset_avant_sorted, all_onsets_sorted)
print("Are trialonset_avant and all_onsets_sorted equal?", are_equal)

# Show differences if not equal
if not are_equal:
    diff_in_avant = np.setdiff1d(trialonset_avant_sorted, all_onsets_sorted)
    diff_in_all = np.setdiff1d(all_onsets_sorted, trialonset_avant_sorted)
    print("In trialonset_avant but not in all_onsets_sorted:", diff_in_avant)
    print("In all_onsets_sorted but not in trialonset_avant:", diff_in_all)

Are trialonset_avant and all_onsets_sorted equal? False
In trialonset_avant but not in all_onsets_sorted: [ 521035 1041709 2053780]
In all_onsets_sorted but not in trialonset_avant: [ 521035.  983423. 1041709. 1178979. 2010387. 2053780. 2852645. 2990011.
 3282244. 3556281. 3695786. 3985515. 4110189. 4554985. 5868678. 6211131.
 6317804. 6470972. 6874274.]


# CSV file


In [213]:

csv_data = pd.read_csv("data/Subject_Session_Selection.csv", sep=";")
csv_data.columns = csv_data.columns.str.strip() 
csv_data.columns

Index(['Mouse Name', 'User (user_userName)', 'Ear tag',
       'Start date (dd.mm.yy)', 'End date', 'Sex_bin', 'strain', 'mutations',
       'Birth date', 'licence', 'DG', 'ExpEnd', 'Created on', 'Session',
       'Session Date (yyymmdd)', 'Start Time (hhmmss)', 'Behavior Type',
       'Session Type', 'Mouse Age (d)', 'Weight of Reference',
       'Weight Session', 'Trial_onset', 'stim_onset', 'catch_onset',
       'Responses_times', 'EMG', 'PtA', 'dCA1', 'mPFC', 'wM1', 'wS1', 'wS2'],
      dtype='object')

In [214]:
csv_data

,Mouse Name,User (user_userName),Ear tag,Start date (dd.mm.yy),End date,Sex_bin,strain,mutations,Birth date,licence,...,stim_onset,catch_onset,Responses_times,EMG,PtA,dCA1,mPFC,wM1,wS1,wS2
0,PL200,Pierre Le Merre,Unknown,19.06.2014,19.06.2014,m,C57BL/6JRj,NaN,Unknown,Unknown,...,25.751;95.532;211.808;318.882;339.278;415.958;...,47.643;72.935;121.426;145.417;163.214;184.412;...,417.655;478.939;590.514;656.5;791.869;817.26;8...,1.4367e-05;4.53e-05;-4.7600000000000005e-05;-1...,2.266e-06;-4.6933999999999995e-05;2.5e-05;1.44...,0.00011766600000000001;0.000107534;0.0001252;0...,0.000100134;8.0066e-05;9.813399999999999e-05;8...,0.000147734;5.24e-05;0.00010920000000000001;0....,4.2134e-05;2.9134e-05;3.4734e-05;3.4734e-05;4....,NaN
1,PL200,Pierre Le Merre,Unknown,20.06.2014,20.06.2014,m,C57BL/6JRj,NaN,Unknown,Unknown,...,20.18;43.172;85.761;241.442;283.329;452.589;64...,71.961;423.095;545.167;665.838;700.83;725.726;...,20.675;43.669;86.258;241.935;283.728;453.086;6...,1.1167e-05;6.433e-06;-1.3667e-05;7.4e-06;3.26e...,-2.5866e-05;-4.0866e-05;-2.3265999999999997e-0...,-0.000133534;-0.000114866;-0.000118134;-9.7866...,-5.8e-05;-3.52e-05;-2.5e-05;-2.0734e-05;-3.613...,-1.4934e-05;-3.26e-05;-6.7534e-05;-7.2934e-05;...,-3e-06;2.2e-06;-1.066e-06;2.334e-06;5.34e-07;5...,NaN
2,PL200,Pierre Le Merre,Unknown,21.06.2014,21.06.2014,m,C57BL/6JRj,NaN,Unknown,Unknown,...,85.187;183.263;228.152;269.345;288.04;445.702;...,18.805;50.293;63.69;111.077;143.271;166.266;37...,85.682;183.86;228.649;269.84;288.737;446.299;4...,-1.8433e-05;5.6067e-05;4.97e-05;4.5267e-05;2.3...,3.4866e-05;5.4134e-05;6.9266e-05;8.2134e-05;7....,-0.00023266600000000003;-0.000221666;-0.000179...,9.313399999999999e-05;7.64e-05;0.0001322000000...,0.000129334;0.000124934;0.000165866;0.0001132;...,4.9133999999999994e-05;6.68e-05;0.000101734;3....,NaN
3,PL200,Pierre Le Merre,Unknown,22.06.2014,22.06.2014,m,C57BL/6JRj,NaN,Unknown,Unknown,...,100.125;127.917;162.909;181.206;242.292;271.18...,76.43;145.711;342.966;367.161;384.355;404.851;...,100.622;128.414;163.308;181.603;242.889;271.58...,-1.6133000000000003e-05;-2.1267e-05;-1.967e-06...,-1.52e-05;-2.86e-05;-2.9266e-05;-3.5066e-05;-3...,1.4934e-05;1.1065999999999999e-05;2.9934e-05;6...,-1.34e-07;6.0934e-05;4.46e-05;4.7066e-05;7.08e...,2.734e-06;-1.46e-05;-1.6934e-05;-5.86660000000...,-0.000130734;-9.76e-05;-9.153400000000001e-05;...,NaN
4,PL200,Pierre Le Merre,Unknown,23.06.2014,23.06.2014,m,C57BL/6JRj,NaN,Unknown,Unknown,...,21.524;97.702;236.669;261.064;276.16;327.451;3...,46.413;72.107;112.399;126.394;149.689;173.383;...,22.119;98.2;237.266;261.563;276.756;328.048;34...,5.9e-06;1.5367e-05;7.433000000000001e-06;-1.39...,-5.1200000000000004e-05;-5.22e-05;-4.6134e-05;...,0.0002002;0.000186734;0.000180934;0.0002038000...,-8.4666e-05;-9.073399999999999e-05;-8.46e-05;-...,-8.6534e-05;-9.7e-05;-9.7334e-05;-5.4733999999...,1.54e-05;1.666e-06;1.8e-06;9.066e-06;1.6134e-0...,NaN
5,PL200,Pierre Le Merre,Unknown,24.06.2014,24.06.2014,m,C57BL/6JRj,NaN,Unknown,Unknown,...,64.182;85.376;102.471;144.261;167.755;182.552;...,24.694;48.381;128.462;247.737;309.321;340.115;...,85.873;102.968;144.66;168.352;183.149;209.244;...,2.9166999999999997e-05;-3.3367e-05;-2.1767e-05...,0.000119066;0.0001192;6.46e-05;9.8334000000000...,-1.1e-05;-1.0933999999999999e-05;-5.1734e-05;-...,-0.0001172;-0.00017353399999999998;-0.00014113...,2e-06;-4.9133999999999994e-05;-0.0001676;-8.2e...,4.4934e-05;3.2666e-05;6.934e-06;2.526600000000...,NaN
6,PL200,Pierre Le Merre,Unknown,25.06.2014,25.06.2014,m,C57BL/6JRj,NaN,Unknown,Unknown,...,47.149;68.54;101.036;122.829;256.698;279.293;3...,23.452;85.835;142.223;157.719;179.812;200.808;...,47.642;69.037;101.633;123.326;257.195;279.888;...,1.3667e-05;1.6329999999999999e-06;2.1e-06;1.80...,-8.04e-05;-7.9334e-05;-4.7466e-05;-7.8734e-05;...,0.00020546600000000001;0.000214266;0.000237066...,-7.4066e-05;-5.1866e-05;-6.346599999999999e-05...,-0.0001292;-8.400000000000001e-05;-6.72e-05;-9...,-2.0266e-05;-2.8266e-05;-1.74e-05;-4.533399999...,

In [218]:
for index, csv_data_row in csv_data.iterrows():
    #display(csv_data_row)
    print(csv_data_row['Mouse Name'])
    print(type(csv_data_row))

PL200
<class 'pandas.core.series.Series'>
PL200
<class 'pandas.core.series.Series'>
PL200
<class 'pandas.core.series.Series'>
PL200
<class 'pandas.core.series.Series'>
PL200
<class 'pandas.core.series.Series'>
PL200
<class 'pandas.core.series.Series'>
PL200
<class 'pandas.core.series.Series'>
PL200
<class 'pandas.core.series.Series'>


# TOTAL CSV

In [212]:
importlib.reload(converters.Initiation_nwb)
output_path = converters.Initiation_nwb.files_to_csv(PL=PL, PLALL=PLALL, csv_file=csv_file)

Number of sessions: 8
Number of files in PLALL: 9
Processing session file: PL200_D1.mat
Processing session file: PL200_D2.mat
Processing session file: PL200_D3.mat
Processing session file: PL200_D4.mat
Processing session file: PL200_D5.mat
Processing session file: PL200_D6.mat
Processing session file: PL200_D7.mat
Processing session file: PL200_D8.mat


 # TOTAL _-_-_-_-_-_-_-_
 

In [223]:

importlib.reload(convert_to_nwb_for_PL)
nwb_path = convert_to_nwb_for_PL.convert_data_to_nwb_pl(csv_file=csv_file, output_folder="data", mouse_name = "PL200")

IndentationError: unexpected indent (convert_to_nwb_for_PL.py, line 103)

# Accessibilité 